# Unzip archivos

# Carga del modelo

In [1]:
from transformers import BertForSequenceClassification, AutoTokenizer

model_directory = '../input/val-data/modelo'

model = BertForSequenceClassification.from_pretrained(model_directory)
tokenizer = AutoTokenizer.from_pretrained('nlpaueb/bert-base-greek-uncased-v1')

Downloading:   0%|          | 0.00/459 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/530k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

# Carga del dataset

In [2]:
import pandas as pd

# OJO que hay textos duplicados
data_path = '../input/val-data/val_df.csv'

df = pd.read_csv(data_path, delimiter='>')

print(df.head())
print(df.describe())

                  Autor                                          Obra  \
0  Eusebius of Caesarea  De Martyribus Palaestinae (Recensio Brevior)   
1  Eusebius of Caesarea  De Martyribus Palaestinae (Recensio Brevior)   
2  Eusebius of Caesarea  De Martyribus Palaestinae (Recensio Brevior)   
3  Eusebius of Caesarea  De Martyribus Palaestinae (Recensio Brevior)   
4  Eusebius of Caesarea  De Martyribus Palaestinae (Recensio Brevior)   

  Fragmento                                              Texto  
0         1  πρωτος τοιγαρουν των επι παλαιστινης μαρτυρων ...  
1         1  πρωτος τοιγαρουν των επι παλαιστινης μαρτυρων ...  
2         2  δεσιου μηνος εβδομη, προ επτα ειδων ιουνιων λε...  
3         3  μετα δε τουτο επι της αυτης πολεως πλειστοι οσ...  
4         4  ομως δε ουν εφερον το αποβαν ακολουθως ταις απ...  
            Autor     Obra Fragmento  \
count        4292     4292      4292   
unique         13       34      1514   
top     Euripides  Bacchae         1   
freq       

In [3]:
df.drop_duplicates(inplace=True, ignore_index=True)
df.head()

,Autor,Obra,Fragmento,Texto
0,Eusebius of Caesarea,De Martyribus Palaestinae (Recensio Brevior),1,πρωτος τοιγαρουν των επι παλαιστινης μαρτυρων ...
1,Eusebius of Caesarea,De Martyribus Palaestinae (Recensio Brevior),1,πρωτος τοιγαρουν των επι παλαιστινης μαρτυρων ...
2,Eusebius of Caesarea,De Martyribus Palaestinae (Recensio Brevior),2,"δεσιου μηνος εβδομη, προ επτα ειδων ιουνιων λε..."
3,Eusebius of Caesarea,De Martyribus Palaestinae (Recensio Brevior),3,μετα δε τουτο επι της αυτης πολεως πλειστοι οσ...
4,Eusebius of Caesarea,De Martyribus Palaestinae (Recensio Brevior),4,ομως δε ουν εφερον το αποβαν ακολουθως ταις απ...


In [4]:
for autor in df.Autor.unique():
    print(autor)
    for obra in df[df.Autor==autor].Obra.unique():
        print('\t'+obra)

        

Eusebius of Caesarea
	De Martyribus Palaestinae (Recensio Brevior)
Demosthenes
	For the Megalopolitans
	Against Evergus and Mnesibulus
	Philippic 2
	Against Onetor
	Philip
Isocrates
	Against Callimachus
	To Philip
Aristophanes
	Acharnians
Theophrastus
	DeLapidibus
	DeNervorumResolutione
Lysias
	For the Soldier
	Against Theomnestus 1
	Against Diogeiton
Aristotle
	deInterpretatione
	Meteorologica
	DePartibusAnimalium
Plutarch
	Cleomenes
Euripides
	Bacchae
Appian
	Sicily and the Other Islands
Hippocrates
	De glandulis
	De muliebribus
	De haemorrhoidibus
	Iusiurandum
	De affectionibus
Plato
	Theages
	Timaeus
	Crito
	Protagoras
Lucian
	Macrobii
	Toxaris vel amicitia
	Deorum Concilium
	De Syria Dea
	Nigrinus


# Interpreter

In [5]:
!pip install transformers-interpret

     |████████████████████████████████| 4.4 MB 955 kB/s eta 0:00:01
  Created wheel for transformers-interpret: filename=transformers_interpret-0.4.0-py3-none-any.whl size=21713 sha256=9a07d510c8809b5f61359518ee863e969d0f67246b70bb89dc4f91a41d68eee6
  Stored in directory: /root/.cache/pip/wheels/dd/91/54/5d4fac93d2c3eea0fb6d7c9af00ea1a3eec72df11cdadf9d67
Successfully built transformers-interpret


In [76]:
# https://github.com/cdpierse/transformers-interpret
from transformers import AutoTokenizer
from transformers_interpret import SequenceClassificationExplainer
import torch
import json
from tqdm import tqdm
from scipy.special import softmax
import numpy as np

In [9]:
with open('../input/val-data/labels_encoder.json', 'r') as f:
    json_dict = f.read()
    label_encoder = json.loads(json_dict)
print(label_encoder)

{'Xenophon': 0, 'Pseudo-Xenophon': 1, 'Hyperides': 2, 'Lycurgus': 3, 'Eusebius of Caesarea': 4, 'Demosthenes': 5, 'Homer': 6, 'Sophocles': 7, 'Isocrates': 8, 'Aristophanes': 9, 'Theophrastus': 10, 'Julian the Emperor': 11, 'Herodotus': 12, 'Strabo': 13, 'Aeschines': 14, 'Pausanias': 15, 'Andocides': 16, 'Antiphon': 17, 'Dinarchus': 18, 'Arrian': 19, 'Callimachus': 20, 'Lysias': 21, 'Apollonius Rhodius': 22, 'Thucydides': 23, 'Philostratus the Athenian': 24, 'Aristotle': 25, 'John, of Damascus (attributed author)': 26, 'Aeschylus': 27, 'Theocritus': 28, 'Apollodorus': 29, 'Plutarch': 30, 'Euripides': 31, 'Polybius': 32, 'Athenaeus': 33, 'Aelian': 34, 'Dionysius of Halicarnassus': 35, 'Procopius': 36, 'Appian': 37, 'Hippocrates': 38, 'Plato': 39, 'Basil, Saint, Bishop of Caesarea': 40, 'Aeneas Tacticus': 41, 'Asclepiodotus': 42, 'Quintus Smyrnaeus': 43, 'Clement of Alexandria': 44, 'Nonnus of Panopolis': 45, 'Cassius Dio Cocceianus': 46, 'Longinus': 47, 'Marcus Aurelius': 48, 'Longus': 4

In [91]:
class TextPredictor():
    def __init__(self, df, text_name, author, model=model, tokenizer=tokenizer, encoder=label_encoder, eval_mode ='interpreter'):
        self.df = df
        self.text = text_name
        self.author = author
        self.model=model
        self.tokenizer = tokenizer
        self.encoder = encoder
        self.chunks = self.fragment()
        self.eval_mode = eval_mode

        if self.eval_mode == 'model':
            self.model = model.cpu()
        elif self.eval_mode == 'interpreter':
            self.model = model.cuda()

        
    def fragment(self):
        text_list = self.df[self.df.Obra == self.text].Texto.to_list()
        text = ' '.join(text_list)
        #print(text)
        splitted_text = text.split()
        n = 100
        text_chunks = [' '.join(splitted_text[i:i+n]) for i in range(0,len(splitted_text),n)]
        return text_chunks

    def predict(self):
        predictions = self.init_prediction_dict()
        for text in tqdm(self.chunks):
            
            if self.eval_mode == 'interpreter':
                interpreter = SequenceClassificationExplainer(self.model, self.tokenizer)
                prediction = interpreter(text)
                #print(f'Prediction: {prediction}\n\n')
                predicted_label=int(interpreter.predicted_class_name.split('_')[-1])
                #print(predicted_label)
                predicted_accuracy = sum([i for _,i in prediction])
                #print(predicted_accuracy)
                #interpreter.visualize()
                for autor,label in self.encoder.items():
                    if label == predicted_label:
                        predictions[autor].append(predicted_accuracy)
                    else:
                        predictions[autor].append(0)
                        
            elif self.eval_mode == 'model':
                tokenized_text = self.tokenizer(text, return_tensors = 'pt')
                prediction = self.model(**tokenized_text)
                prediction = prediction.logits
                prediction = prediction.detach()
                prediction = prediction.numpy()[0]
                prediction = softmax(prediction)
                
                for autor, label in self.encoder.items():
                    predictions[autor].append(prediction[label])
    
        self.show_predictions(predictions)            
            
            
    def show_predictions(self, predictions, top=5):
        print(f'Text: {self.text}')
        print('True author: '+self.author)
        confidence_dict = self.confidence(predictions)
        pred_autor = max(confidence_dict, key=confidence_dict.get)
        pred_conf = confidence_dict[pred_autor]
        print(f'\tPrediction: {pred_autor}\n\tConfidence: {round(pred_conf,3)}%')
        
        del confidence_dict[pred_autor]
        
        for i in range(top-1):
            pred_autor = max(confidence_dict, key=confidence_dict.get)
            pred_conf = confidence_dict[pred_autor]
            print(f'\t\tAlternative prediction: {pred_autor}\n\t\tConfidence: {round(pred_conf,3)}%')
            del confidence_dict[pred_autor]

    def init_prediction_dict(self):
        pred_dict = {}
        for autor in self.encoder.keys():
            pred_dict[autor] = []
        return pred_dict
    
    def confidence(self, predictions):
        '''
        Ver si tiene sentido hacerlo para interpreter y model igual
        '''
        predictions_temp = {} # {Autor: val}
    
        for autor, list_prob in predictions.items():
            if self.eval_mode == 'interpreter':
                predictions_temp[autor] = sum(list_prob)
            elif self.eval_mode == 'model':
                predictions_temp[autor] = np.mean(list_prob)
        
        sum_conf = sum(predictions_temp.values())
        
        predictions_final = {autor: conf/sum_conf * 100 for autor, conf in predictions_temp.items()}
                
        return predictions_final

In [46]:
confidence_dict = {'Plato': 8, 'MiMadre': 15, 'yo':0.4, 'david':8000}

pred_autor = max(confidence_dict, key=confidence_dict.get)

print(pred_autor)

david


In [94]:
text_pred_interpreter = TextPredictor(df, 'DePartibusAnimalium', 'Aristotle', eval_mode='interpreter')
text_pred_model = TextPredictor(df, 'DePartibusAnimalium', 'Aristotle', eval_mode='model')

print('MODEL')
text_pred_model.predict()
print('INTERPRETER')
text_pred_interpreter.predict()

  0%|          | 0/659 [00:00<?, ?it/s]

Text: DePartibusAnimalium
True author: Aristotle
	Prediction: Theophrastus
	Confidence: 61.501%
		Alternative prediction: Aristotle
		Confidence: 32.122%
		Alternative prediction: Dio Chrysostom
		Confidence: 2.026%
		Alternative prediction: Demetrius of Phaleron (attributed author)
		Confidence: 1.651%
		Alternative prediction: Lucian
		Confidence: 0.628%


100%|██████████| 659/659 [09:04<00:00,  1.21it/s]

Text: DePartibusAnimalium
True author: Aristotle
	Prediction: Theophrastus
	Confidence: 72.333%
		Alternative prediction: Aristotle
		Confidence: 23.338%
		Alternative prediction: Dio Chrysostom
		Confidence: 2.027%
		Alternative prediction: Demetrius of Phaleron (attributed author)
		Confidence: 1.492%
		Alternative prediction: Lucian
		Confidence: 0.416%


In [48]:
for autor in df.Autor.unique():
    for obra in df[df.Autor==autor].Obra.unique():
        TextPredictor(df, obra, autor, model, tokenizer, label_encoder).predict()

  0%|          | 0/19 [00:00<?, ?it/s]

Text: De Martyribus Palaestinae (Recensio Brevior)
True author: Eusebius of Caesarea
	Prediction: Eusebius of Caesarea
	Confidence: 97.3554076634425%
		Alternative prediction: Lucian
		Confidence: 1.5152965703934482%
		Alternative prediction: John, of Damascus (attributed author)
		Confidence: 0.6081721847442297%
		Alternative prediction: Julian the Emperor
		Confidence: 0.5211235814198191%
		Alternative prediction: Xenophon
		Confidence: 0.0%


  0%|          | 0/54 [00:00<?, ?it/s]

Text: For the Megalopolitans
True author: Demosthenes
	Prediction: Demosthenes
	Confidence: 82.9405293235216%
		Alternative prediction: Isocrates
		Confidence: 9.69086173851403%
		Alternative prediction: Dio Chrysostom
		Confidence: 4.304572606917286%
		Alternative prediction: Andocides
		Confidence: 3.0640363310470877%
		Alternative prediction: Xenophon
		Confidence: 0.0%


  0%|          | 0/20 [00:00<?, ?it/s]

...

Text: Against Callimachus
True author: Isocrates
	Prediction: Lysias
	Confidence: 100.0%
		Alternative prediction: Xenophon
		Confidence: 0.0%
		Alternative prediction: Pseudo-Xenophon
		Confidence: 0.0%
		Alternative prediction: Hyperides
		Confidence: 0.0%
		Alternative prediction: Lycurgus
		Confidence: 0.0%


  0%|          | 0/66 [00:00<?, ?it/s]

Text: To Philip
True author: Isocrates
	Prediction: Isocrates
	Confidence: 100.0%
		Alternative prediction: Xenophon
		Confidence: 0.0%
		Alternative prediction: Pseudo-Xenophon
		Confidence: 0.0%
		Alternative prediction: Hyperides
		Confidence: 0.0%
		Alternative prediction: Lycurgus
		Confidence: 0.0%


 97%|█████████▋| 64/66 [01:02<00:01,  1.02it/s]


KeyboardInterrupt: 